In [5]:
# list of candidates in the order listed on the voting site
candidates = ["Brandon Allgood (Valo Heath)", "Prasanth Pulavarthi (Microsoft)", "Alexandre Eichenberger (IBM)", "Rajeev Nalawdi (Intel)", "Sonal Agrawal (Binary Fountain)", "Wenming Ye (AWS)", "Mayank Kaushik (NVIDIA)"]

# votes with company (added manually), email address, and votes ranking for each candidate
votes = [
    ["MS", "1", 8,	1,	3,	2,	8,	8,	8], 
    ["MS", "1", 8,	1,	8,	8,	8,	8,	8], 	
    ["MS", "1", 7,	1,	2,	3,	6,	4,	5], 
    ["MS", "1", 7,	1,	2,	5,	6,	4,	3], 
    ["MS", "1", 6,	5,	5,	5,	7,	5,	5], 
    ["MS", "1", 6,	1,	5,	4,	7,	3,	2], 
    ["MS", "1", 2,	1,	2,	2,	3,	2,	2], 
    ["MS", "1", 6,	1,	4,	2,	7,	5,	3],
    ["IBM", "1", 7,	2,	1,	5,	7,	5,	5], 
    ["IBM", "1", 7,	1,	2,	3,	6,	5,	4], 
    ["IBM", "1", 7,	7,	1,	7,	7,	7,	7], 
    ["IBM", "1", 7,	1,	2,	3,	7,	5,	4], 
    ["IBM", "1", 7,	2,	1,	3,	7,	4,	4], 
    ["IBM", "1", 2,	3,	1,	6,	5,	7,	4],
    ["NVIDIA", "1", 6,	2,	3,	5,	7,	4,	1],
    ["NVIDIA", "1", 3,	2,	3,	3,	3,	3,	1],
    ["FACEBOOK", "1", 7,	1,	2,	2,	7,	2,	2],
    ["MIT", "1", 8,	8,	1,	8,	8,	8,	8],
    ["amazon", "1", 4,	2,	3,	4,	4,	1,	4],
    ["pinduoduo", "1", 5,	7,	5,	5,	5,	6,	5],
    ["walgreens", "1", 4,	1,	2,	3,	6,	7,	5]
]

In [6]:
import numpy as np

# from the list of all ballots, seclect the ones for this company (or all if company is "ALL")
def ballot_for_company(ballots, select_company, verbose=0):
    res = []
    for b in ballots:
        curr_company, curr_ballot = b[0], b[2:]
        if select_company == 'ALL' or curr_company == select_company:
            res.append(curr_ballot)
            if verbose:
                print(curr_company, curr_ballot)
    return np.array(res)

# create a new ballot reflecting the choices in the given ballot sample
def rank(ballots, candidates, verbose=0):
    cnum = candidates.size
    #m[a,b] is "time folks preferred candidate b over a"
    m = np.zeros((cnum, cnum), dtype=int)
    for b in ballots:
        for j in range(cnum):
            for k in range(cnum):
                if b[k]>b[j]:
                    m[k][j] = m[k][j] + 1
    total = np.sum(m, axis=0) # sum over columns
    # (candidate id, preference score) ranked by most preferred first
    rank = sorted(zip(range(cnum),total), reverse=True, key=lambda record: record[1])
    # new ballot: where candidates that have the same preferrence have the same score (low is better)
    new_ballot = np.zeros(cnum, dtype=int)
    preference = 0
    previous_score = 10000000
    for p in range(cnum): # by order of decreasing popularity
        curr_candidate_id, curr_candidate_score = rank[p]
        if curr_candidate_score != previous_score:
            preference +=1 # different score, advance by 1
        new_ballot[curr_candidate_id] = preference
        previous_score = curr_candidate_score   
    if verbose:
        print("Matrix m[a, b], times voters preferred candidate b over a:")
        for i in range(cnum):
            print(i, ": ", m[i])
        print("Totals (sum over columns of m) =", total)
        print("Rank (candidate id, candidate preference score) = ", rank)
        print("New ballot = ", new_ballot)
    return new_ballot

# create a ballot that reflects all the votes per company
def rank_per_company(votes, candidates, verbose=0):
    companies = np.unique(votes[:, 0])
    ballot_per_company = []
    for com in companies:
        print("Ballot for company = ", com)
        new_ballot = rank(ballot_for_company(votes, com), candidates, verbose=verbose)
        print("  is ", new_ballot)
        ballot_per_company.append(new_ballot)
        
    final_ballot = rank(np.array(ballot_per_company), candidates, verbose=verbose)
    if verbose:
        print("Final ballot = ", final_ballot)
    return final_ballot

# translate a ballot into the list of ordered candidates using names
def ballot_to_candidates(ballot, candidates):
    cnum = candidates.size
    # (candidate id, order) ranked by most preferred first
    rank = sorted(zip(range(cnum),ballot), key=lambda record: record[1])
    for r in rank:
        print(r[1], ":", candidates[r[0]])

In [8]:
# use numpy arrays
vv = np.array(votes)
cc = np.array(candidates)

#ballot_for_company(vv, 'INTEL')
#rank(ballot_for_company(vv, "INTEL"), cc, verbose=1)

# actual vote
final_ballot_per_company = rank_per_company(vv, cc)
print("\nFinal ballot (summarizing per company)=", final_ballot_per_company)
ballot_to_candidates(final_ballot_per_company, cc)

# control vote
ballot_per_voter = rank(ballot_for_company(vv, "ALL"), cc)
print("\nBallot per voter =", ballot_per_voter)
ballot_to_candidates(ballot_per_voter, cc)

Ballot for company =  FACEBOOK
  is  [3 1 2 2 3 2 2]
Ballot for company =  IBM
  is  [6 2 1 3 7 5 4]
Ballot for company =  MIT
  is  [2 2 1 2 2 2 2]
Ballot for company =  MS
  is  [5 1 2 2 6 4 3]
Ballot for company =  NVIDIA
  is  [6 2 3 5 7 4 1]
Ballot for company =  amazon
  is  [4 2 3 4 4 1 4]
Ballot for company =  pinduoduo
  is  [1 3 1 1 1 2 1]
Ballot for company =  walgreens
  is  [4 1 2 3 6 7 5]

Final ballot (summarizing per company)= [4 1 1 2 5 3 2]
1 : Prasanth Pulavarthi (Microsoft)
1 : Alexandre Eichenberger (IBM)
2 : Rajeev Nalawdi (Intel)
2 : Mayank Kaushik (NVIDIA)
3 : Wenming Ye (AWS)
4 : Brandon Allgood (Valo Heath)
5 : Sonal Agrawal (Binary Fountain)

Ballot per voter = [6 1 2 4 7 5 3]
1 : Prasanth Pulavarthi (Microsoft)
2 : Alexandre Eichenberger (IBM)
3 : Mayank Kaushik (NVIDIA)
4 : Rajeev Nalawdi (Intel)
5 : Wenming Ye (AWS)
6 : Brandon Allgood (Valo Heath)
7 : Sonal Agrawal (Binary Fountain)
